In [1]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install easypip")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("bbrl>=0.2.2")
easyinstall("swig")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("bbrl_gymnasium[box2d]")
easyinstall("bbrl_gymnasium[classic_control]")
easyinstall("tensorboard")
easyinstall("moviepy")
easyinstall("box2d-kengz")


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import sys
from pathlib import Path
import math
import time

from moviepy.editor import ipython_display as video_display
import time
from tqdm.auto import tqdm
from typing import Tuple, Optional
from functools import partial

import torch
import bbrl_gymnasium

import copy
from abc import abstractmethod, ABC
import torch.nn as nn
import torch.nn.functional as F
from time import strftime

In [3]:
from datetime import datetime
from datetime import date

import base64
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

import gymnasium
from gymnasium import logger as gymlogger
# from gym.wrappers import Monitor # deprecated 2023 - https://stackoverflow.com/questions/71520568/importerror-cannot-import-name-monitor-from-gym-wrappers
from gymnasium.wrappers.record_video import RecordVideo
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")

def show_video(loop=True, num=0):
    mp4list = glob.glob(f'videoTest/rl-video-episode-{num}.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        if loop == True:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
        else:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    
def wrap_env(env):
    env = RecordVideo(env, './videoTest',  episode_trigger = lambda episode_number: True) # !!! 2023
    env.reset() # !!! 2023
    #env = Monitor(env, './video', force=True)
    return env


 2024-01-17 14:08:22 GMT
OK.


In [26]:
"""
Continuous action version of the classic cart-pole system implemented by Rich
Sutton et al.
"""
import math

import numpy as np

from gymnasium.envs.classic_control.cartpole import CartPoleEnv
from gymnasium import logger, spaces
from gymnasium.wrappers import TimeLimit

import sys
sys.modules[__name__]


"""
Continuous action version of the classic cart-pole system implemented by Rich
Sutton et al.
"""

class ContinuousEnvCACLA(CartPoleEnv):
    """Continuous version  of the CartPole-v1 environment"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.min_action = -1.0
        self.max_action = 1.0
        self.action_space = spaces.Box(
            self.min_action, self.max_action, shape=(1,), dtype=np.float64
        )

    def step(self, action):
        if action > self.max_action:
            action = np.array(self.max_action)
        elif action < self.min_action:
            action = np.array(self.min_action)
        assert self.state is not None, "Call reset before using step method."

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag * float(action)

        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (
            force + self.polemass_length * theta_dot**2 * sintheta
        ) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == "euler":
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        terminated = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            # Pole just fell!
            self.steps_beyond_terminated = 0
            reward = 1.0
        else:
            if self.steps_beyond_terminated == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), reward, terminated, False, {}

class ContinuousEnvArticle(ContinuousEnvCACLA):
    """Continuous version  of the CartPole-v1 environment"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.min_action = -1
        self.max_action = 1

    def reset(
        self,
        *,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
    ):
        super().reset(seed=seed)
        # Note that if you use custom reset bounds, it may lead to out-of-bound
        # state/observations.
        x = self.np_random.uniform(low=-0.05, high=0.05, size=(1,))
        self.state = 0, 0, x, 0
        self.steps_beyond_terminated = None

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), {}
    
try:
  gymnasium.envs.register(
      id='CartpoleEnvCacla',
      entry_point='__main__:ContinuousEnvCACLA',
      max_episode_steps=500
  )
except:
    print("Except")
    pass
try:
  gymnasium.envs.register(
      id='CartpoleEnvArticle',
      entry_point='__main__:ContinuousEnvArticle',
      max_episode_steps=500
  )
except:
    print("Except")
    pass


In [27]:
from tqdm import tqdm
from torch.nn.functional import mse_loss
from copy import copy
class Cacla(nn.Module):
    def __init__(self, in_dim, 
                 h_dim, activation=nn.Tanh,
                 discount_factor=0.95, 
                 gaussian_noise=0.01, 
                 var=False, 
                 exploration='gaussian',
                 transform= lambda x: x,
                transform_critic = lambda x: x,
                transform_actor =  lambda x: x):
        super(Cacla,self).__init__()
        self.critic = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1),
        )
        
        self.actor = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1),
        )
        # transform
        self.transform = transform
        self.transform_critic = transform_critic
        self.transform_actor = transform_actor
        # learning hyperparaters
        self.discount_factor = discount_factor
        self.vart = 1
        self.beta = 0.001
        self.with_var = var

        # exploration parameters
        self.gaussian_noise = gaussian_noise
        self.eps = 1
        self.eps_decay = 0.99
        self.eps_min = 0.01
        self.exploration = 'gaussian'
    
    def forward(self, x):
        x = self.transform(x)
        x = torch.tensor(x).unsqueeze(0)
        return self.transform_critic(self.critic(x)), self.transform_actor(self.actor(x))
    
    def actor_forward(self, x):
        x = self.transform(x)
        x = torch.from_numpy(x).unsqueeze(0)
        return self.transform_actor(self.actor(x))
                            
    def critic_forward(self, x):
        x = self.transform(x)
        x = torch.from_numpy(x).unsqueeze(0)
        return self.transform_critic(self.critic(x))

    # Loss computation
    def compute_critic_loss(self, value, target):
        delta = (target - value).detach()
        loss =  mse_loss(value, target)
        n_update = 1
        if self.with_var and delta > 0:
            n_update = torch.ceil(delta/np.sqrt(self.vart)).item()
            self.vart = (1-self.beta)*self.vart +self.beta*loss.detach()
        return loss, int(n_update)
    
    def compute_actor_loss(self, value, target):
        return mse_loss(value, target)
    
    # Exploration
    def sample(self, x):
        if self.exploration == 'gaussian':
            return self.sample_gaussian(x)
        elif self.exploration == 'epsilon':
            return self.sample_epsilon(x)
        else: 
            print('Exploration unknown: ', self.exploration)       
    
    def sample_gaussian(self, x):
        return (x + np.random.normal(0,self.gaussian_noise))
    
    def sample_epsilon(self, x, inf=-1, sup=1):
        if torch.rand(1) > self.eps: return x
        else: return np.random.rand() * (sup - inf) - inf


def test(eval_env, model, n_test = 10, noise_std = 0.3):
    cum_reward = 0
    with torch.no_grad():
        for _ in range(n_test):
            done = False
            truncated = False
            obs0,_ = eval_env.reset()
            while not done and not truncated:
                a0 = model.actor_forward(obs0)  
                obs0, reward, done,truncated,_ = eval_env.step(a0) 
                cum_reward += reward
    return cum_reward/n_test

def addnoise(x, std):
    return x + np.random.normal(0,std)
    
def train(eval_env, 
          train_env,
          model,
          optim, 
          step_max=102400, 
          step_eval=1024, 
          n_test=10, 
          noise_std=0.3):
    obs ,_ = train_env.reset()
    scores = []
    optim_critic, optim_actor = optim
    count = 0
    # boucle d'apprentissage
    for it in range(step_max+1):
        v0, a0 = model(obs)
        obs0 = obs
        action = copy(a0)
        action = model.sample(action).detach()
        obs, reward, done, truncated, reste = train_env.step(action)
        
        # Compute losses
        with torch.no_grad(): v1 = model.critic_forward(obs)
        target_v = addnoise(reward, noise_std) + model.discount_factor*v1
        optim_critic.zero_grad()
        critic_loss, n_update = model.compute_critic_loss(v0, target=target_v)
        critic_loss.backward()
        optim_critic.step()
        if (target_v - v0).item() > 0: 
            for _ in range(n_update):
                optim_actor.zero_grad()
                mse_loss(model.actor_forward(obs), action).backward()
                optim_actor.step()
        if done or truncated: obs,_ = train_env.reset()
        #if it % (100*(1<<count)) == 0:
        if it % 1024 == 0:
            #if it % step_eval == 0:
            count += 1
            perf = test(eval_env, model, n_test, noise_std)
            scores.append((it, perf))
            print(f'{it = } | reward {perf}')
    return scores



In [28]:
from torch.optim import SGD, Adam
from functools import partial
# Initialisation

def make_env(env_name, max_episode_steps= 500):
    return TimeLimit(gymnasium.make(env_name), max_episode_steps)

# Make env
env_name = 'CartpoleEnvArticle'
#env_name = 'CartpoleEnvArticle'
max_episode = 500
train_env = make_env(env_name, max_episode_steps=max_episode)
eval_env = make_env(env_name, max_episode_steps=max_episode)

all_perfs = []
n_runs = 20

# Model hyperparameters
discount_factor = 0.95
noise_std= 0.3
exploration = 'gaussian'
var = False

path = f'log/{exploration}/32_var_normobs_gamma{int(discount_factor*100)}_std{noise_std}'

obsmax = np.array([0.21, 2.14, 0.27, 3.32], dtype=np.float32)
vmax, vmin = 5, 5
amax, amin = -1, 1

def norm_obs(x, xmax):
    return x / xmax

def norm_out(v, vmin, vmax):
    return 2*(v - vmin) / (vmax-vmax)-1

if not os.path.exists(path): 
    os.mkdir(path)

for run in range(1,n_runs+1):
    print(f'======= {run = } / {n_runs} =========')
    cacla = Cacla(in_dim=4, h_dim=32, 
                  discount_factor=discount_factor, 
                  gaussian_noise=0.1,
                  activation = nn.Tanh,
                 var=var,
                 exploration=exploration,
                 #transform=partial(norm_obs, xmax=obsmax),
                 #transform_critic = partial(norm_out, vmin=vmin, vmax=vmax),
                 #transform_actor = partial(norm_out, vmin=amin, vmax=amax)
                 )
    optim_crit = SGD(cacla.critic.parameters(), lr=0.01)
    optim_act = SGD(cacla.actor.parameters(), lr=0.01)
    optim = (optim_crit, optim_act)
    scores = train(eval_env, train_env, cacla, optim, 
                   step_max=102400,
                  n_test=10,
                  noise_std= noise_std)
    scores = np.array(scores)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    np.savetxt(path+f'/{timestr}.log', scores)

======= run = 1 / 20 =========
it = 0 | reward 30.3
it = 1024 | reward 18.2
it = 2048 | reward 24.3
it = 3072 | reward 43.1
it = 4096 | reward 62.6
it = 5120 | reward 71.7
it = 6144 | reward 55.6
it = 7168 | reward 83.3
it = 8192 | reward 59.8
it = 9216 | reward 61.9
it = 10240 | reward 62.8
it = 11264 | reward 54.0
it = 12288 | reward 72.0
it = 13312 | reward 70.4
it = 14336 | reward 74.5
it = 15360 | reward 84.8
it = 16384 | reward 89.6
it = 17408 | reward 81.9
it = 18432 | reward 64.1
it = 19456 | reward 77.0
it = 20480 | reward 66.9
it = 21504 | reward 69.9
it = 22528 | reward 69.8
it = 23552 | reward 86.9
it = 24576 | reward 53.6
it = 25600 | reward 63.9
it = 26624 | reward 54.8
it = 27648 | reward 70.7
it = 28672 | reward 81.5
it = 29696 | reward 68.8
it = 30720 | reward 88.2
it = 31744 | reward 54.9
it = 32768 | reward 42.4
it = 33792 | reward 58.0
it = 34816 | reward 60.9
it = 35840 | reward 100.9
it = 36864 | reward 89.6
it = 37888 | reward 45.6
it = 38912 | reward 74.7
it = 3

KeyboardInterrupt: 

In [ ]:
# Attention, ce sont des bornes empiriques
print(f'Les bornes actions sont {cacla.actbornes} et les bornes etat sont {cacla.bornes} et bornes critic{cacla.vbornes}')

In [ ]:
rec_env = wrap_env(TimeLimit(ContinuousEnvCACLA(render_mode='rgb_array'),max_episode_steps=500))
observation, _ = rec_env.reset()
maxEvaluations = 2
evaluation = 0
score = []
total = 0
while evaluation < maxEvaluations:
    rec_env.render()
    _, a0 = cacla(observation)  
    action = a0
    observation, reward, done, truncated, _ = rec_env.step(action) 
    total += reward
    if done or truncated:
      score.append(total)
      total = 0
      evaluation = evaluation + 1
      observation, _ = rec_env.reset()
rec_env.close()
score

In [ ]:
show_video(num=1)

## Visualisation

In [ ]:
def get(path):
    X = []
    for data_file in os.listdir(path):
        filepath = path + data_file 
        X.append(np.loadtxt(filepath)[:,:1227])
    return np.array(X)
#path = 'log/gaussian/gamma90/'


## CACLA

In [ ]:
X = get('./log/gaussian/now_novar_gamma90_std0.3/')
X.shape

In [ ]:
for x in X:
    plt.plot(x[:,0], x[:,1], alpha=1)
    #break
plt.title(f'CACLA+Var: gamme: {discount_factor} | noise_std: {noise_std}')
plt.xlabel('training iterations')
plt.ylabel('reward cumulée')
plt.show()

In [ ]:
x = X.mean(0)
plt.plot(x[:,0], x[:,1])
plt.title(f'CACLA+Var: gamme: {discount_factor} | noise_std: {noise_std}')
plt.xlabel('training iterations')
plt.ylabel('reward cumulée')
plt.show()

## CACLA + VAR

In [ ]:
X = get('./log/gaussian/now_gamma90_std0.3/')
X.shape

In [ ]:
for x in X[:5]:
    plt.plot(x[:,0], x[:,1], alpha=1)
    #break
plt.title(f'CACLA+Var: gamme: {discount_factor} | noise_std: {noise_std}')
plt.xlabel('training iterations')
plt.ylabel('reward cumulée')
plt.show()

In [ ]:
x = X.mean(0)
plt.plot(x[:,0], x[:,1])
plt.title(f'CACLA+Var: gamme: {discount_factor} | noise_std: {noise_std}')
plt.xlabel('training iterations')
plt.ylabel('reward cumulée')
plt.show()

In [ ]:
from tqdm import tqdm
from torch.nn.functional import mse_loss
class Cacla(nn.Module):
    def __init__(self, in_dim, 
                 h_dim, activation=nn.Tanh,
                 discount_factor=0.95, 
                 gaussian_noise=0.01, 
                 var=False, 
                 exploration='gaussian'):
        super(Cacla,self).__init__()
        self.critic = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1),
            #activation()
        )
        
        self.actor = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1),
        )
        
        self.discount_factor = discount_factor
        self.gaussian_noise = gaussian_noise
        self.vart = 1
        self.beta = 0.001
        self.with_var = var

        self.eps = 1
        self.eps_decay = 0.99
        self.eps_min = 0.01
        
        self.exploration = 'gaussian'
        # ordre : xmax, xmin, xprimemax, xprimemin, thetamax, thetamin, thetaprimemax, thetaprimemin
        self.bornes = np.array([np.inf for i in range(8)])
        self.actbornes = np.array([np.inf for i in range(2)])
        self.vbornes = np.array([np.inf,np.inf])
    
    def forward(self, x, testing=False):
        x = torch.from_numpy(x).unsqueeze(0)
        if testing: return (None, self.actor(x)) 
        
        res = self.critic(x), self.actor(x)
        return res
    
    # Loss computation
    def compute_critic_loss(self, value, target):
        delta = (target - value).detach()
        loss =  mse_loss(value, target)
        n_update = 1
        if self.with_var and delta > 0:
            n_update = torch.ceil(delta/np.sqrt(self.vart)).item()
            self.vart = (1-self.beta)*self.vart +self.beta*loss.detach()
        return loss, int(n_update)
    
    def compute_actor_loss(self, value, target):
        return mse_loss(value, target)
    
    # Exploration
    def sample(self, x):
        if self.exploration == 'gaussian':
            return self.sample_gaussian(x)
        elif self.exploration == 'epsilon':
            return self.sample_epsilon(x)
        else: 
            print('Exploration unknown: ', self.exploration)       
    
    def sample_gaussian(self, x):
        return (x + np.random.normal(0,self.gaussian_noise))
    
    def sample_epsilon(self, x, inf=-1, sup=1):
        if torch.rand(1) > self.eps: return x
        else: return np.random.rand() * (sup - inf) - inf
        
    # Mise à jour des facteurs pour la normalisation
    def updateB(self,x,theta,xprime,thetaprime):
        # Le cas initial
        if self.bornes[0] == np.inf :
            self.bornes[0] = x
            self.bornes[1] = x
            self.bornes[2] = xprime
            self.bornes[3] = xprime
            self.bornes[4] = theta
            self.bornes[5] = theta
            self.bornes[6] = thetaprime
            self.bornes[7] = thetaprime
        
        # update x, xprime, theta et thetaprime
        self.bornes[0] = np.max((self.bornes[0],x))
        self.bornes[1] = np.min((self.bornes[1],x))
        
        self.bornes[2] = np.max((self.bornes[2],xprime))
        self.bornes[3] = np.min((self.bornes[3],xprime))
        
        self.bornes[4] = np.max((self.bornes[4],theta))
        self.bornes[5] = np.min((self.bornes[5],theta))
        
        self.bornes[6] = np.max((self.bornes[6],thetaprime))
        self.bornes[7] = np.min((self.bornes[7],thetaprime))

def makenorm():
    env_name = 'CartpoleEnvCacla'
    num_samples = 1000
    env = make_env(env_name)
    samples = np.array([env.reset()[0] for _ in range(num_samples)])
    min_values = np.min(samples, axis=0)
    max_values = np.max(samples, axis=0)
    return min_values, max_values

def addnoise(x, std):
    return x + np.random.normal(0,std)

def normalize(x):
    normact = 2*((x+0.77478176)/(0.85380614 + 0.77478176))-1
    #print("Norm act ",normact)
    return normact

obsmax = np.array([0.21, 2.14, 0.27, 3.32], dtype=np.float32)
bornes = np.array([2.45187998, -2.45674968,  
                   0.23211133, -0.23395817,  
                   3.21429443, -3.3032372, 
                   1.78533256, -1.56042111])
def normalize_obs(x):
    return x/obsmax

def normalize_critic(x):
    return 2*((x+0.19350152)/(11.54955292 + 0.19350152))-1
    

def test(eval_env, model, n_test = 10, noise_std = 0.3):
    cum_reward = 0
    with torch.no_grad():
        for _ in range(n_test):
            done = False
            truncated = False
            obs0,_ = eval_env.reset()
            while not done and not truncated:
                # normalisation
                obs0 = normalize_obs(obs0)
                _, a0 = model(obs0, testing=True)
                
                obs0, reward, done,truncated,_ = eval_env.step(normalize(a0))
                
                cum_reward += reward
    return cum_reward/n_test
    
def train(eval_env, 
          train_env,
          model,
          optim, 
          eval_interval=1000, 
          step_max=100000, 
          n_test=10, 
          noise_std=0.3):
    obs ,_ = train_env.reset()

    
    model.updateB(obs[0],obs[1],obs[2],obs[3])
    scores = []
    optim_critic, optim_actor = optim
    # Extra pour récupérer des bornes
    # boucle d'apprentissage
    for it in range(step_max):
        # quelques normalisations lineaires
        obs = normalize_obs(obs)
        v0, a0 = model(obs)
        a0 = normalize(a0)
        #v0 = normalize_critic(v0)

        obs0 = obs
        action = model.sample(a0).detach()
        #action = normalize(action)
        # max - min
        
        obs, reward, done, truncated,reste = train_env.step(addnoise(action, noise_std))
        
        #model.updateB(obs[0],obs[1],obs[2],obs[3])
        
        # Compute losses
        with torch.no_grad(): v1, _ = model(obs)
        target_v = addnoise(reward, noise_std) + model.discount_factor*v1
        optim_critic.zero_grad()
        critic_loss, n_update = model.compute_critic_loss(v0, target=target_v)
        critic_loss.backward()
        optim_critic.step()
        if target_v - v0 > 0: 
            for _ in range(n_update):
                optim_actor.zero_grad()
                mse_loss(normalize(model(obs0)[1]), action).backward()
                optim_actor.step()
        if done or truncated: obs,_ = train_env.reset()
        if it % eval_interval == 0:
            perf = test(eval_env, model, n_test, noise_std)
            scores.append((it, perf))
            print(f'{it = } | reward {perf}')
    return scores





In [ ]:
from torch.optim import SGD, Adam
# Initialisation

def make_env(env_name, max_episode_steps= 500):
    return TimeLimit(gymnasium.make('CartpoleEnvCacla'), max_episode_steps)

env_name = 'CartpoleEnvCacla'
max_episode = 500
train_env = make_env(env_name, max_episode_steps=max_episode)
eval_env = make_env(env_name, max_episode_steps=max_episode)

all_perfs = []
n_runs = 1
discount_factor = 0.9
noise_std= 0.3
exploration = 'gaussian'
var = False
path = f'log/{exploration}/n_novar_gamma{int(discount_factor*100)}_std{noise_std}'

if not os.path.exists(path): 
    print()
    os.mkdir(path)

for run in range(1,n_runs+1):
    print(f'======= {run = } / {n_runs} =========')
    cacla = Cacla(in_dim=4, h_dim=12, 
                  discount_factor=discount_factor, 
                  gaussian_noise=0.1,
                 var=var,
                 exploration=exploration)
    optim_crit = SGD(cacla.critic.parameters(), lr=0.01)
    optim_act = SGD(cacla.actor.parameters(), lr=0.01)
    optim = (optim_crit, optim_act)
    scores = train(eval_env, train_env, cacla, optim, 
                   eval_interval=1024, 
                   step_max=102400,
                  n_test=10,
                  noise_std= noise_std)
    scores = np.array(scores)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    np.savetxt(path+f'/{timestr}.log', scores)

In [ ]:
env_test = make_env('CartpoleEnvArticle')
xmax = np.zeros(4)
for _ in range(100):
    done = False
    truncated = False
    obs0,_ = env_test.reset()
    xmax = np.maximum(xmax, np.abs(obs0))
    while not done and not truncated:
        obs0, reward, done,truncated,_ = env_test.step(1)
        xmax = np.maximum(xmax, np.abs(obs0))
xmax.round(2)
